In [121]:
#Processing evcc block ack
import csv
import numpy as np

with open('csv/DataLink/max_interf_VO/evcc_blockack_max_VO.csv', newline='') as csvfile:
    rows = csv.DictReader(csvfile)
    startint_seq_num_block_ack = []
    timelist_block_ack = []
    
    for row in rows:
        startint_seq_num_block_ack.append(int(row['Starting Sequence Number']))
        timelist_block_ack.append(float(row['Time']))

In [122]:
#for debug
if(len(startint_seq_num_block_ack) != len(timelist_block_ack)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_block_ack))

1106


In [123]:
#Processing evcc send
with open('csv/DataLink/max_interf_VO/evcc_send_max_VO_add_missing_SDPreq.csv', newline='') as csvfile:
    rows = csv.DictReader(csvfile)
    timelist_evcc_send = []
    seq_num_evcc_send = []
    data_length_evcc_send = []
    info_evcc_send = []
    frame_num_evcc_send = []
    
    for row in rows:
        timelist_evcc_send.append(float(row['Time']))
        seq_num_evcc_send.append(int(row['Sequence number']))
        data_length_evcc_send.append(int(row['Data Length']))
        info_evcc_send.append(str(row['Info']))
        frame_num_evcc_send.append(int(row['No.']))

In [124]:
#for debug
if(len(timelist_evcc_send) != len(seq_num_evcc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send) != len(data_length_evcc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send) != len(info_evcc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send) != len(frame_num_evcc_send)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_evcc_send))

1386


In [125]:
timelist_evcc_send_split = []
seq_num_evcc_send_split = []
data_length_evcc_send_split = []
info_evcc_send_split = []
frame_num_evcc_send_split = []

round_start_point = 0
round_end_point = 0

sdpreq_index_list = []
#Fetch first packet index of data length = 74 (SDP request) in each round
for i in range(0, len(data_length_evcc_send)):
    if(i == 0 and data_length_evcc_send[i] == 74):
        sdpreq_index_list.append(i)
    if(i > 0 and data_length_evcc_send[i] == 74 and data_length_evcc_send[i-1] != 74):
        sdpreq_index_list.append(i)
sdpreq_index_list.append(len(data_length_evcc_send)) #Add the end of list index

for i in range(0, len(sdpreq_index_list)-1):
    timelist_evcc_send_split.append([timelist_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])
    seq_num_evcc_send_split.append([seq_num_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])
    data_length_evcc_send_split.append([data_length_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])
    info_evcc_send_split.append([info_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])
    frame_num_evcc_send_split.append([frame_num_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])

In [126]:
#for debug
if(len(timelist_evcc_send_split) != len(seq_num_evcc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send_split) != len(data_length_evcc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send_split) != len(info_evcc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send_split) != len(frame_num_evcc_send_split)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_evcc_send_split)) #Output # of testing rounds

20


In [127]:
#for debug, this is ideally the time of each testing round
for i in range(1, len(timelist_evcc_send_split)):
    print(timelist_evcc_send_split[i][0]-timelist_evcc_send_split[i-1][0])

21.429094
20.781616999999997
20.856195
21.299105999999995
21.198332000000022
20.993989
22.11468099999999
20.991749
20.93880999999999
21.352163999999988
20.479962999999998
21.91763900000001
20.965844000000004
20.710294000000033
22.21826599999997
20.41248200000001
21.315517999999997
22.168479999999988
21.298812999999996


In [128]:
#Set evcc time anchor in each round per stage
evcc_send_time_anchor = []
#data_length_in_each_stage = [74, 96, 189, 173, 173, 173, 185, 461, 189, 173]
wierd_packet = {}
packet_loss_testing_round_7_flag = False
packet_loss_testing_round_8_flag = False
packet_loss_testing_round_15_flag = False

global last_SeqNum

last_SeqNum = 0

def printinfo(stageNum, frameNum, info_string):
    print("-----------------------------------------")
    print("Stage " + str(stageNum) + " finish. No. " + str(frameNum) + ", Info " + str(info_string))

def print_time_difference(evcc_send_time_anchor_each_round):
    print("Time difference: " + str(evcc_send_time_anchor_each_round[len(evcc_send_time_anchor_each_round)-1]-
                          evcc_send_time_anchor_each_round[len(evcc_send_time_anchor_each_round)-2]))
    
def checkstage_num(stageNum, valid_stageNum, info_string, time_string, datalen, seqnum, frameNum):
    global last_SeqNum
    
    if(stageNum == valid_stageNum):
        check_wield_packet_list(frameNum)
        
        #Avoid 802.11 retry by sequence number
        if(seqnum == last_SeqNum):
            return False
        else:
            last_SeqNum = seqnum
        return True
    
    else:
        #debug
        print("\nFalse")
        print("Stage Num = " + str(stageNum))
        print("Valid Stage Num = " + str(valid_stageNum))
        print("What is this packet? Data Length = " + str(datalen) + " No. " + str(frameNum))
        print("Time " + str(time_string) + " #Seq " + str(seqnum) + " Info " + str(info_string) + "\n")
        wierd_packet[frameNum] = [datalen, stageNum]
        return False

def check_wield_packet_list(frameNum):
    if(wierd_packet.get(frameNum) != None):
        del wierd_packet[frameNum]
        print("\nNo. " + str(frameNum) + " has been found. Delete from wierd list.\n")
    
for i in range(0, len(data_length_evcc_send_split)):
#for i in [17]:
    print("\n-------- Test Round " + str(i+1) + " --------")
    #Stage 1
    evcc_send_time_anchor_each_round = [timelist_evcc_send_split[i][0]]
    stageNum = 1
    printinfo(stageNum, frame_num_evcc_send_split[i][0], info_evcc_send_split[i][0])
    stageNum = stageNum + 1
    
    for j in range(1, len(data_length_evcc_send_split[i])):
        
        #Deal with packet loss......
        if(i == 15 and packet_loss_testing_round_15_flag == False):
            evcc_send_time_anchor_each_round.append(373.708951)
            printinfo(stageNum, 1944425, "Packet Loss")
            print_time_difference(evcc_send_time_anchor_each_round)
            stageNum = stageNum + 1
            packet_loss_testing_round_15_flag = True
        
        #Stage 2
        if(data_length_evcc_send_split[i][j] == 96 and data_length_evcc_send_split[i][j-1] != 96):
            if(checkstage_num(stageNum, 2, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 3
        if(data_length_evcc_send_split[i][j] == 189 and data_length_evcc_send_split[i][j-1] != 189):
            if(checkstage_num(stageNum, 3, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                #Deal with packet loss......
                if(i == 7 and packet_loss_testing_round_7_flag == False):
                    evcc_send_time_anchor_each_round.append(204.698447)
                    printinfo(stageNum, 1041301, "Packet Loss")
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    packet_loss_testing_round_7_flag = True
                
                continue
                
        #Stage 4 
        if(data_length_evcc_send_split[i][j] == 173 and data_length_evcc_send_split[i][j-1] != 173):
            
            #if(info_evcc_send_split[i][j][-2:] == 'TC'):
            if(checkstage_num(stageNum, 4, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                continue
           
        #Stage 5, tricky
        if(data_length_evcc_send_split[i][j] == 173):    
            if(info_evcc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 5, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                    evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                    printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    continue
            
            else: #info_evcc_send_split[i][j][-2:] == 'FC'
                #Backward checking 2 packets
                isTCfound = False
                if(i == 1):
                    for k in range(1, 4):
                        if(data_length_evcc_send_split[i][j-k] == 173 and info_evcc_send_split[i][j-k][-2:] == 'TC'):
                            isTCfound = True
                            break
                else:
                    for k in range(1, 3):
                        if(data_length_evcc_send_split[i][j-k] == 173 and info_evcc_send_split[i][j-k][-2:] == 'TC'):
                            isTCfound = True
                            break
                
                if(not isTCfound):
                    if(checkstage_num(stageNum, 5, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                        print("Stage " + str(stageNum) +
                            ", Not TC? No. " + str(frame_num_evcc_send_split[i][j]) +
                            ", Data length " + str(data_length_evcc_send_split[i][j]) +
                            ", Info " + str(info_evcc_send_split[i][j]) + " , Count FC because TC cannot be found.")
                        evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                        printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                        print_time_difference(evcc_send_time_anchor_each_round)
                        stageNum = stageNum + 1  
                        
                        continue
                
        #Stage 6, tricky
        if(data_length_evcc_send_split[i][j] == 173):
            if(info_evcc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 6, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                    evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                    printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1

                    continue

            else: #info_evcc_send_split[i][j][-2:] == 'FC'
                #Backward checking 2 packets
                isTCfound = False
                for k in range(1, 3):
                    if(data_length_evcc_send_split[i][j-k] == 173 and info_evcc_send_split[i][j-k][-2:] == 'TC'):
                        isTCfound = True
                        break

                if(not isTCfound):
                    if(checkstage_num(stageNum, 6, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                        print("Stage " + str(stageNum) +
                            ", Not TC? No. " + str(frame_num_evcc_send_split[i][j]) +
                            ", Data length " + str(data_length_evcc_send_split[i][j]) +
                            ", Info " + str(info_evcc_send_split[i][j]) + " , Count FC because TC cannot be found.")
                        evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                        printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                        print_time_difference(evcc_send_time_anchor_each_round)
                        stageNum = stageNum + 1
                        
                        #Deal with packet loss......
                        if(i == 8 and packet_loss_testing_round_8_flag == False):
                            evcc_send_time_anchor_each_round.append(226.234073)
                            printinfo(stageNum, 1161238, "Packet Loss")
                            print_time_difference(evcc_send_time_anchor_each_round)
                            stageNum = stageNum + 1
                            packet_loss_testing_round_8_flag = True

                        continue

        #Stage 7
        if(data_length_evcc_send_split[i][j] == 185 and data_length_evcc_send_split[i][j-1] != 185):
            if(checkstage_num(stageNum, 7, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 8
        if(data_length_evcc_send_split[i][j] == 461 and data_length_evcc_send_split[i][j-1] != 461):
            if(checkstage_num(stageNum, 8, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
        #Stage 9
        if(data_length_evcc_send_split[i][j] == 189 and data_length_evcc_send_split[i][j-1] != 189):
            if(checkstage_num(stageNum, 9, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 10
        if(data_length_evcc_send_split[i][j] == 173 and data_length_evcc_send_split[i][j-1] != 173):
            
            if(info_evcc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 10, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                    evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                    printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
            else: #info_evcc_send_split[i][j][-2:] == 'FC'
                #Backward checking 2 packets
                isTCfound = False
                for k in range(1, 3):
                    if(data_length_evcc_send_split[i][j-k] == 173 and info_evcc_send_split[i][j-k][-2:] == 'TC'):
                        isTCfound = True
                        break

                if(not isTCfound):
                    if(checkstage_num(stageNum, 10, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                        print("Stage " + str(stageNum) +
                            ", Not TC? No. " + str(frame_num_evcc_send_split[i][j]) +
                            ", Data length " + str(data_length_evcc_send_split[i][j]) +
                            ", Info " + str(info_evcc_send_split[i][j]) + " , Count FC because TC cannot be found.")
                        evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                        printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                        print_time_difference(evcc_send_time_anchor_each_round)
                        stageNum = stageNum + 1
                
        if(stageNum == 11): break
        
    evcc_send_time_anchor.append(evcc_send_time_anchor_each_round)


-------- Test Round 1 --------
-----------------------------------------
Stage 1 finish. No. 222626, Info QoS Data, SN=1772, FN=0, Flags=.p.....TC
-----------------------------------------
Stage 2 finish. No. 226031, Info QoS Data, SN=1927, FN=0, Flags=.p....F.C
Time difference: 0.6288040000000024
-----------------------------------------
Stage 3 finish. No. 228968, Info QoS Data, SN=1781, FN=0, Flags=.p.....TC
Time difference: 0.4939189999999982

False
Stage Num = 4
Valid Stage Num = 3
What is this packet? Data Length = 189 No. 229104
Time 56.168195 #Seq 1942 Info QoS Data, SN=1942, FN=0, Flags=.p....F.C


False
Stage Num = 4
Valid Stage Num = 9
What is this packet? Data Length = 189 No. 229104
Time 56.168195 #Seq 1942 Info QoS Data, SN=1942, FN=0, Flags=.p....F.C

-----------------------------------------
Stage 4 finish. No. 229731, Info QoS Data, SN=1783, FN=0, Flags=.p.....TC
Time difference: 0.17730000000000246

False
Stage Num = 5
Valid Stage Num = 4
What is this packet? Data Le

In [129]:
#for debug
print(str(len(evcc_send_time_anchor)) + '\n')
for i in range(0, len(evcc_send_time_anchor)):
    print("i = " + str(i))
    print(len(evcc_send_time_anchor[i]))

20

i = 0
10
i = 1
10
i = 2
10
i = 3
10
i = 4
10
i = 5
10
i = 6
10
i = 7
10
i = 8
10
i = 9
10
i = 10
6
i = 11
10
i = 12
10
i = 13
10
i = 14
10
i = 15
10
i = 16
10
i = 17
6
i = 18
10
i = 19
10


In [130]:
#wierd_packet
#len(wierd_packet)

In [131]:
#for debug
for i in range(0, len(evcc_send_time_anchor)):
    print("i = " + str(i))
    print(evcc_send_time_anchor[i][len(evcc_send_time_anchor[i])-1]-evcc_send_time_anchor[i][0])

i = 0
2.003536000000004
i = 1
1.8959449999999975
i = 2
1.9220040000000012
i = 3
2.1720810000000057
i = 4
2.2122610000000122
i = 5
1.816158999999999
i = 6
2.7784369999999967
i = 7
1.9206350000000043
i = 8
1.918419
i = 9
2.1111460000000193
i = 10
1.7517530000000079
i = 11
2.5369790000000307
i = 12
2.1192849999999908
i = 13
1.4743609999999876
i = 14
2.461724000000004
i = 15
1.878479000000027
i = 16
2.120075999999983
i = 17
2.260218000000009
i = 18
2.0652150000000233
i = 19
1.9275719999999978


In [132]:
#Processing secc send
with open('csv/DataLink/max_interf_VO/secc_send_max_VO_add_missing_SDPres.csv', newline='') as csvfile:
    rows = csv.DictReader(csvfile)
    timelist_secc_send = []
    seq_num_secc_send = []
    data_length_secc_send = []
    info_secc_send = []
    frame_num_secc_send = []
    
    for row in rows:
        timelist_secc_send.append(float(row['Time']))
        seq_num_secc_send.append(int(row['Sequence number']))
        data_length_secc_send.append(int(row['Data Length']))
        info_secc_send.append(str(row['Info']))
        frame_num_secc_send.append(int(row['No.']))

In [133]:
#for debug
if(len(timelist_secc_send) != len(seq_num_secc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send) != len(data_length_secc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send) != len(info_secc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send) != len(frame_num_secc_send)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_secc_send))

1069


In [134]:
timelist_secc_send_split = []
seq_num_secc_send_split = []
data_length_secc_send_split = []
info_secc_send_split = []
frame_num_secc_send_split = []

round_start_point = 0
round_end_point = 0

sdpres_index_list = []
#Fetch first packet index of data length = 92 (SDP response) in each round
for i in range(0, len(data_length_secc_send)):
    if(i == 0 and data_length_secc_send[i] == 92):
        sdpres_index_list.append(i)
    if(i > 0 and data_length_secc_send[i] == 92 and data_length_secc_send[i-1] != 92):
        sdpres_index_list.append(i)
sdpres_index_list.append(len(data_length_secc_send)) #Add the end of list index

for i in range(0, len(sdpres_index_list)-1):
        timelist_secc_send_split.append([timelist_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])
        seq_num_secc_send_split.append([seq_num_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])
        data_length_secc_send_split.append([data_length_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])
        info_secc_send_split.append([info_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])
        frame_num_secc_send_split.append([frame_num_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])

In [135]:
#for debug
if(len(timelist_secc_send_split) != len(seq_num_secc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send_split) != len(data_length_secc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send_split) != len(info_secc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send_split) != len(frame_num_secc_send_split)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_secc_send_split)) #Output # of testing rounds

20


In [136]:
for i in range(0, len(timelist_secc_send_split)):
    print('i = ' + str(i))
    print(len(timelist_secc_send_split[i]))

i = 0
50
i = 1
47
i = 2
54
i = 3
54
i = 4
56
i = 5
55
i = 6
49
i = 7
52
i = 8
52
i = 9
57
i = 10
40
i = 11
59
i = 12
54
i = 13
55
i = 14
56
i = 15
56
i = 16
53
i = 17
48
i = 18
72
i = 19
50


In [137]:
#for debug, this is ideally the time of each testing round
for i in range(1, len(timelist_secc_send_split)):
    print(timelist_secc_send_split[i][0]-timelist_secc_send_split[i-1][0])

21.412377999999997
20.771749
20.790840000000003
21.305191000000008
21.186722000000003
20.997915999999975
22.11174200000002
20.995535999999987
20.988957999999997
21.298581999999982
20.478169000000037
22.072833000000003
20.831750999999997
20.70017899999999
22.213572999999997
20.47129000000001
21.23957999999999
22.182292000000018
21.295533999999975


In [138]:
for i in range(0, len(timelist_secc_send_split)):
    print('i = ' + str(i) + ' ', end = '')
    print(len(timelist_secc_send_split[i]))

i = 0 50
i = 1 47
i = 2 54
i = 3 54
i = 4 56
i = 5 55
i = 6 49
i = 7 52
i = 8 52
i = 9 57
i = 10 40
i = 11 59
i = 12 54
i = 13 55
i = 14 56
i = 15 56
i = 16 53
i = 17 48
i = 18 72
i = 19 50


In [139]:
for i in range(1, len(timelist_secc_send_split)):
    print(timelist_secc_send_split[i][0]-timelist_secc_send_split[i-1][0])

21.412377999999997
20.771749
20.790840000000003
21.305191000000008
21.186722000000003
20.997915999999975
22.11174200000002
20.995535999999987
20.988957999999997
21.298581999999982
20.478169000000037
22.072833000000003
20.831750999999997
20.70017899999999
22.213572999999997
20.47129000000001
21.23957999999999
22.182292000000018
21.295533999999975


In [140]:
#Set secc time anchor in each round per stage
secc_send_time_anchor = []
#data_length_in_each_stage = [92, 173, 157, 189, 253, 173, 189, 173, 493, 173]
wierd_packet_secc = {}
packet_loss_testing_round_6_flag = False
packet_loss_testing_round_15_flag = False
packet_loss_testing_round_16_flag = False
packet_loss_testing_round_19_flag = False

global last_seqNum
last_seqNum = 0

def printinfo(stageNum, frameNum, info_string):
    print("-----------------------------------------")
    print("Stage " + str(stageNum) + " finish. No. " + str(frameNum) + ", Info " + str(info_string))

def print_time_difference(secc_send_time_anchor_each_round):
    print("Time difference: " + str(secc_send_time_anchor_each_round[len(secc_send_time_anchor_each_round)-1]-
                          secc_send_time_anchor_each_round[len(secc_send_time_anchor_each_round)-2]))
    
def checkstage_num(stageNum, valid_stageNum, info_string, time_string, datalen, seqnum, frameNum):
    global last_seqNum
    
    if(stageNum == valid_stageNum):
        if(seqnum == last_seqNum):
            return False
        else:
            check_wield_packet_list(frameNum)
            return True
    else:
        #debug
        print("\nWhat is this packet? Data Length = " + str(datalen) + " No. " + str(frameNum))
        print("Time " + str(time_string) + " #Seq " + str(seqnum) + " Info " + str(info_string) + "\n")
        wierd_packet_secc[frameNum] = [datalen, stageNum]
        return False

def check_wield_packet_list(frameNum):
    if(wierd_packet_secc.get(frameNum) != None):
        del wierd_packet_secc[frameNum]
        print("\nNo. " + str(frameNum) + " has been found. Delete from wierd list.\n")
    
for i in range(0, len(data_length_secc_send_split)):
#for i in [16, 19]:
    print("\n-------- Test Round " + str(i+1) + " --------")
    #Stage 1
    secc_send_time_anchor_each_round = [timelist_secc_send_split[i][0]]
    stageNum = 1
    printinfo(stageNum, frame_num_secc_send_split[i][0], info_secc_send_split[i][0])
    stageNum = stageNum + 1

    for j in range(1, len(data_length_secc_send_split[i])):
        #Stage 2
        if(data_length_secc_send_split[i][j] == 173 and data_length_secc_send_split[i][j-1] != 173):
            if(info_secc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 2, info_secc_send_split[i][j],
                                      timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                      seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                    secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                    printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    continue
                    
            else:
                print("Stage " + str(stageNum) +
                      ", Not TC? No. " + str(frame_num_secc_send_split[i][j]) +
                      ", Data length " + str(data_length_secc_send_split[i][j]) +
                      ", Info " + str(info_secc_send_split[i][j]))
        #Stage 3
        if(data_length_secc_send_split[i][j] == 157 and data_length_secc_send_split[i][j-1] != 157):
            if(checkstage_num(stageNum, 3, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 4
        if(data_length_secc_send_split[i][j] == 189 and data_length_secc_send_split[i][j-1] != 189):
            if(checkstage_num(stageNum, 4, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                    
                continue
                
        #Stage 5
        if(data_length_secc_send_split[i][j] == 253 and data_length_secc_send_split[i][j-1] != 253):
            if(checkstage_num(stageNum, 5, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                
                #Deal with packet loss......
                if(i == 19 and packet_loss_testing_round_19_flag == False):
                    secc_send_time_anchor_each_round.append(459.774772)
                    printinfo(stageNum, 2449519, "Packet Loss")
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    packet_loss_testing_round_19_flag = True
                
                continue
                
        #Stage 6
        if(data_length_secc_send_split[i][j] == 173 and data_length_secc_send_split[i][j-1] != 173):
            if(info_secc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 6, info_secc_send_split[i][j],
                                      timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                      seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                    secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                    printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    #Deal with packet loss......
                    if(i == 15 and packet_loss_testing_round_15_flag == False):
                        secc_send_time_anchor_each_round.append(374.622105)
                        printinfo(stageNum, 1949761, "Packet Loss")
                        print_time_difference(secc_send_time_anchor_each_round)
                        stageNum = stageNum + 1
                        packet_loss_testing_round_15_flag = True
                        
                    continue
                    
            else:
                print("Stage " + str(stageNum) +
                      ", Not TC? No. " + str(frame_num_secc_send_split[i][j]) +
                      ", Data length " + str(data_length_secc_send_split[i][j]) +
                      ", Info " + str(info_secc_send_split[i][j]))
        #Stage 7
        if(data_length_secc_send_split[i][j] == 189 and data_length_secc_send_split[i][j-1] != 189):
            if(checkstage_num(stageNum, 7, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 8
        if(data_length_secc_send_split[i][j] == 173 and data_length_secc_send_split[i][j-1] != 173):
            if(info_secc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 8, info_secc_send_split[i][j],
                                      timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                      seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                    secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                    printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    #Deal with packet loss......
                    if(i == 6 and packet_loss_testing_round_6_flag == False):
                        secc_send_time_anchor_each_round.append(184.132436)
                        printinfo(stageNum, 920468, "Packet Loss")
                        print_time_difference(secc_send_time_anchor_each_round)
                        stageNum = stageNum + 1
                        
                        secc_send_time_anchor_each_round.append(184.463523)
                        printinfo(stageNum, 922698, "Packet Loss")
                        print_time_difference(secc_send_time_anchor_each_round)
                        stageNum = stageNum + 1
                        packet_loss_testing_round_1_flag = True
                    
                    #Deal with packet loss......
                    if(i == 16 and packet_loss_testing_round_16_flag == False):
                        secc_send_time_anchor_each_round.append(395.728053)
                        printinfo(stageNum, 2065078, "Packet Loss")
                        print_time_difference(secc_send_time_anchor_each_round)
                        stageNum = stageNum + 1
                        packet_loss_testing_round_16_flag = True
                    
                    continue
                    
            else:
                print("Stage " + str(stageNum) +
                      ", Not TC? No. " + str(frame_num_secc_send_split[i][j]) +
                      ", Data length " + str(data_length_secc_send_split[i][j]) +
                      ", Info " + str(info_secc_send_split[i][j]))
        #Stage 9
        if(data_length_secc_send_split[i][j] == 493 and data_length_secc_send_split[i][j-1] != 493):
            if(checkstage_num(stageNum, 9, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 10
        if(data_length_secc_send_split[i][j] == 173 and data_length_secc_send_split[i][j-1] != 173):
            if(info_secc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 10, info_secc_send_split[i][j],
                                      timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                      seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                    secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                    printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    continue
                    
            else:
                print("Stage " + str(stageNum) +
                      ", Not TC? No. " + str(frame_num_secc_send_split[i][j]) +
                      ", Data length " + str(data_length_secc_send_split[i][j]) +
                      ", Info " + str(info_secc_send_split[i][j]))
        
        if(stageNum == 11): break
        
    secc_send_time_anchor.append(secc_send_time_anchor_each_round)


-------- Test Round 1 --------
-----------------------------------------
Stage 1 finish. No. 223125, Info QoS Data, SN=3475, FN=0, Flags=.p.....TC
-----------------------------------------
Stage 2 finish. No. 228697, Info QoS Data, SN=3484, FN=0, Flags=.p.....TC
Time difference: 0.9826929999999976
-----------------------------------------
Stage 3 finish. No. 229512, Info QoS Data, SN=3486, FN=0, Flags=.p.....TC
Time difference: 0.15255200000000002
-----------------------------------------
Stage 4 finish. No. 229884, Info QoS Data, SN=3488, FN=0, Flags=.p.....TC
Time difference: 0.07162600000000197
-----------------------------------------
Stage 5 finish. No. 230113, Info QoS Data, SN=3490, FN=0, Flags=.p.....TC
Time difference: 0.05212999999999823

What is this packet? Data Length = 173 No. 230577
Time 56.487665 #Seq 3492 Info QoS Data, SN=3492, FN=0, Flags=.p.....TC


No. 230577 has been found. Delete from wierd list.

-----------------------------------------
Stage 6 finish. No. 230

In [141]:
#for debug
print(str(len(secc_send_time_anchor)) + '\n')
for i in range(0, len(secc_send_time_anchor)):
    print(str(i) + ": " + str(len(secc_send_time_anchor[i])))

20

0: 10
1: 10
2: 10
3: 10
4: 10
5: 10
6: 10
7: 10
8: 10
9: 10
10: 3
11: 10
12: 10
13: 10
14: 10
15: 10
16: 10
17: 3
18: 10
19: 10


In [142]:
#for debug
for i in range(0, len(secc_send_time_anchor)):
    print(secc_send_time_anchor[i][len(secc_send_time_anchor[i])-1]-secc_send_time_anchor[i][0])

1.9516449999999992
1.8188200000000023
1.8585659999999962
2.1667029999999983
2.204004999999995
1.8130599999999788
2.8657160000000204
1.9196450000000027
1.9171080000000131
2.1196680000000185
0.8066380000000208
2.5585849999999937
2.002256999999986
1.460047999999972
2.4603700000000117
1.8702729999999974
2.096734999999967
0.8771990000000187
2.061183999999969
1.9400100000000293


In [143]:
#BLOCK ACK processing
#Time evcc sends blockACK after secc send
evcc_blockAck_time_anchor = []

blockack_index = 0
for i in range(0, len(secc_send_time_anchor)):
    evcc_blockAck_time_anchor_in_each_round = []
    
    for j in range(0, len(secc_send_time_anchor[i])):
        while(timelist_block_ack[blockack_index] < secc_send_time_anchor[i][j]):
            blockack_index = blockack_index + 1
        evcc_blockAck_time_anchor_in_each_round.append(timelist_block_ack[blockack_index])
        #print("ACK time difference = " + str(
        #timelist_block_ack[blockack_index] - secc_send_time_anchor[i][j]))
    
    evcc_blockAck_time_anchor.append(evcc_blockAck_time_anchor_in_each_round)

In [144]:
#SECC Packet Loss dict & Block Ack Loss
#secc_packet_loss_dict[i] = {stageNum}
# i = 5, j = 0 is packet loss of evcc BLOCK ACK
# i = 3, j = 9 is packet loss of evcc BLOCK ACK
secc_packet_loss_dict = {}
secc_packet_loss_dict[6] = {8, 9}
secc_packet_loss_dict[15] = {6}
secc_packet_loss_dict[16] = {8}
secc_packet_loss_dict[19] = {5}

#Block ack loss
secc_packet_loss_dict[9] = {0}
secc_packet_loss_dict[14] = {5}


#for i in secc_packet_loss_dict.get(5): print(i)
#7 in secc_packet_loss_dict.get(5)

In [145]:
#Calculate the avg. block ack latency
sum_latency = 0
cnt = 0
for i in range(0, len(evcc_blockAck_time_anchor)):
    for j in range(0, len(evcc_blockAck_time_anchor[i])):
        if(secc_packet_loss_dict.get(i) == None):
            sum_latency = sum_latency + (evcc_blockAck_time_anchor[i][j]-secc_send_time_anchor[i][j])
            cnt = cnt + 1
        elif(secc_packet_loss_dict.get(i) != None):
            if(j in secc_packet_loss_dict.get(i)): continue
            else:
                sum_latency = sum_latency + (evcc_blockAck_time_anchor[i][j]-secc_send_time_anchor[i][j])
                cnt = cnt + 1
        else: continue
avg_blockAck_latency = sum_latency/cnt
print("Avg Block Ack latency = " + str(avg_blockAck_latency) + " s")
print(cnt)

Avg Block Ack latency = 0.04514084357541716 s
179


In [146]:
#For curiousity
#Calculate the avg. block ack latency of the appending packet loss packets
sum_latency_2 = 0
cnt = 0
for i in range(0, len(evcc_blockAck_time_anchor)):
    for j in range(0, len(evcc_blockAck_time_anchor[i])):
        if(secc_packet_loss_dict.get(i) != None):
            if(j in secc_packet_loss_dict.get(i)):
                sum_latency_2 = sum_latency_2 + (evcc_blockAck_time_anchor[i][j]-secc_send_time_anchor[i][j])
                cnt = cnt + 1
        else: continue
avg_blockAck_latency_2 = sum_latency_2/cnt
print("Avg Block Ack latency = " + str(avg_blockAck_latency_2) + " s")
print(cnt)

Avg Block Ack latency = 0.12235585714285792 s
7


In [147]:
#For debug. Can see testing round 6 (i = 5) has longer block ack latency due to packet loss.
# I only append the secc send time, but didn't append the block time.
from statistics import mean

for i in range(0, len(evcc_blockAck_time_anchor)):
    print("i = " + str(i) + ": " + str(mean(evcc_blockAck_time_anchor[i])-mean(secc_send_time_anchor[i])))

i = 0: 0.020196699999999623
i = 1: 0.03487649999999576
i = 2: 0.028250500000012835
i = 3: 0.042009700000008365
i = 4: 0.05136179999999513
i = 5: 0.02928810000000226
i = 6: 0.059341200000005756
i = 7: 0.03705449999998223
i = 8: 0.02548580000001266
i = 9: 0.04873239999997736
i = 10: 0.0088923333333355
i = 11: 0.06348189999999931
i = 12: 0.025841600000035214
i = 13: 0.013804500000048847
i = 14: 0.11359659999999394
i = 15: 0.037318800000036845
i = 16: 0.05235690000000659
i = 17: 0.3322623333332899
i = 18: 0.05616370000001325
i = 19: 0.05216259999997419


In [148]:
#Use the avg. block ack latency for the lost secc sent packets
for i in range(0, len(evcc_blockAck_time_anchor)):
    for j in range(0, len(evcc_blockAck_time_anchor[i])):
        if(secc_packet_loss_dict.get(i) != None):
            if(j in secc_packet_loss_dict.get(i)):
                evcc_blockAck_time_anchor[i][j] = secc_send_time_anchor[i][j] + avg_blockAck_latency
                print("i = " + str(i))
                print("j = " + str(j))
        else: continue

i = 6
j = 8
i = 6
j = 9
i = 9
j = 0
i = 14
j = 5
i = 15
j = 6
i = 16
j = 8
i = 19
j = 5


In [149]:
#DEL Test Round 11, 18

for i in [17, 10]:
    del evcc_send_time_anchor[i]
    del secc_send_time_anchor[i]
    del evcc_blockAck_time_anchor[i]


#for debug
print(str(len(evcc_send_time_anchor)) + '\n')
for i in range(0, len(evcc_send_time_anchor)):
    print(len(evcc_send_time_anchor[i]))
print('\n')    
    
#for debug
print(str(len(secc_send_time_anchor)) + '\n')
for i in range(0, len(secc_send_time_anchor)):
    print(len(secc_send_time_anchor[i]))
print('\n')
    
#for debug
print(str(len(evcc_blockAck_time_anchor)) + '\n')
for i in range(0, len(evcc_blockAck_time_anchor)):
    print(len(evcc_blockAck_time_anchor[i]))

18

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


18

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


18

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


In [150]:
#for debug check wierd value
for i in range(0, len(evcc_blockAck_time_anchor)):
    for j in range(0, len(evcc_blockAck_time_anchor[i])):
        if(secc_send_time_anchor[i][j] - evcc_send_time_anchor[i][j] < 0):
            print("i = " + str(i) + " j = " + str(j))
        if(evcc_blockAck_time_anchor[i][j] - evcc_send_time_anchor[i][j] < 0):
            print("i = " + str(i) + " j = " + str(j))

In [151]:
#For debug
i = 9
for j in [0]:
    print("Stage " + str(j+1))
    print(evcc_send_time_anchor[i][j])
    print(secc_send_time_anchor[i][j])
    print(evcc_blockAck_time_anchor[i][j])
    print(evcc_blockAck_time_anchor[i][j]-evcc_send_time_anchor[i][j])

Stage 1
245.62783
245.694043
245.7391838435754
0.11135384357541511


In [152]:
path = 'time_record/max_VO_80211.txt'
f = open(path, 'w')
for i in range(0, len(evcc_send_time_anchor)):
    for j in range(0, len(evcc_send_time_anchor[i])):
        print((evcc_blockAck_time_anchor[i][j] - evcc_send_time_anchor[i][j])*1000, file=f, end="")
        f.writelines('\t')
    f.writelines('\t')
    print((evcc_blockAck_time_anchor[i][1] - evcc_send_time_anchor[i][0])*1000, file=f, end="")
    f.writelines('\t')
    print((evcc_blockAck_time_anchor[i][len(evcc_blockAck_time_anchor[i])-1] - evcc_send_time_anchor[i][0])*1000, file=f, end="")
    f.writelines('\t')
    f.writelines('20')
    f.writelines('\t')
    f.writelines('0')
    f.writelines('\t')
    f.writelines('\n')
f.close()

In [153]:
#如果 TCP 有 retransmission，在 Data Link Layer seq 會往下傳送，很難抓取